In [1]:
import pandas as pd
import numpy as np
import re

# Load Data

In [2]:
Data = pd.read_csv('All Reviews1.csv', encoding = "ISO-8859-1" )

In [3]:
Data.head(5)

,Unnamed: 0,Review
0,0,Update: Now that the tech issues have been add...
1,1,Changing my rating since the problems are fixe...
2,2,App worked pretty much as expected. Could have...
3,3,Highly recommend when you travel with Air Cana...
4,4,The app went from very useful to absolutely us...


In [4]:
Data.shape

(1763, 2)

In [5]:
Data.duplicated().value_counts()

False    1763
dtype: int64

In [6]:
sentances = Data['Review']

In [7]:
sentances

0       Update: Now that the tech issues have been add...
1       Changing my rating since the problems are fixe...
2       App worked pretty much as expected. Could have...
3       Highly recommend when you travel with Air Cana...
4       The app went from very useful to absolutely us...
                              ...                        
1758    Terrible to find the flight,  information onli...
1759    Worst App again.... Automatically shuts down. ...
1760    My flight information just dissapears during l...
1761    This app just randomly loses your data, and cr...
1762          Boarding passes only show up half the time.
Name: Review, Length: 1763, dtype: object

# Text Wrangling and Normalisation

In [8]:
from bs4 import BeautifulSoup
import re
import tqdm
import unicodedata

In [9]:
#function defined to remove html tags from data

def strip_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  [s.extract() for s in soup(['iframe', 'script'])]
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
  return stripped_text

In [10]:
#function defined to remove unicode data

def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

In [11]:
# lower case and remove special characters\whitespaces

def pre_process_corpus(docs):
  norm_docs = []
  for doc in tqdm.tqdm(docs):
    doc = strip_html_tags(doc)
    doc = doc.translate(doc.maketrans("\n\t\r", "   "))
    doc = doc.lower()
    doc = remove_accented_chars(doc)
    #doc = contractions.fix(doc)
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = re.sub(' +', ' ', doc)
    doc = doc.strip()  
    norm_docs.append(doc)
  
  return norm_docs

In [12]:
norm_reviews = pre_process_corpus(sentances)

100%|██████████| 1763/1763 [00:00<00:00, 6998.77it/s]


In [13]:
norm_reviews

['update now that the tech issues have been addressed changing to 4 stars from 1 star earlier the checkin functionality is great and also like to call out ability to view incoming flights often 34 flights down need to have altitude status and progression integrated into the app',
 'changing my rating since the problems are fixed in new update good work air canada',
 'app worked pretty much as expected could have used terminal maps to know where we were going for our next gate',
 'highly recommend when you travel with air canada air canada rouge or ac express',
 'the app went from very useful to absolutely useless to useful again',
 'finally fixed thank you air canada',
 'i really like the previous interface it was a big upgrade from the old app and yet the new interface had some welcome user friendly ui additions what i dont understand is why some useful features were taken away such as integrating itinerary into the calendar app and being able to change paymentpersonal info in the app

# Word 2 Vec

In [30]:
import os
import smart_open
import gensim
from gensim.models import Word2Vec

In [15]:
model = gensim.models.Word2Vec(sentences=norm_reviews, workers=4,iter=10,min_count=3)

In [16]:
model.corpus_count

1763

In [28]:
from nltk.cluster import KMeansClusterer
import nltk
from sklearn import cluster
from sklearn import metrics

In [37]:
# get vector data
#model[model.wv]
#rint (X)
 
print (list(model.build_vocab_from_freq))
 
#print (len(list(model.vocab)))

TypeError: 'method' object is not iterable

In [41]:
from sklearn.decomposition import LatentDirichletAllocation

In [42]:
LDA_cv = LatentDirichletAllocation(n_components=6,random_state=30)

In [43]:
LDA_cv.fit(model)

ValueError: Expected 2D array, got scalar array instead:
array=<function model at 0x000001F10CDD6708>.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.